In [66]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 12: Subterranean Sustainability ---</h2><p>The year 518 is significantly more underground than your history books implied.  Either that, or you've arrived in a <span title="It's probably this one. Can never be too sure, though.">vast cavern network</span> under the North Pole.</p>
<p>After exploring a little, you discover a long tunnel that contains a row of small pots as far as you can see to your left and right.  A few of them contain plants - someone is trying to grow things in these geothermally-heated caves.</p>
<p>The pots are numbered, with <code>0</code> in front of you.  To the left, the pots are numbered <code>-1</code>, <code>-2</code>, <code>-3</code>, and so on; to the right, <code>1</code>, <code>2</code>, <code>3</code>.... Your puzzle input contains a list of pots from <code>0</code> to the right and whether they do (<code>#</code>) or do not (<code>.</code>) currently contain a plant, the <em>initial state</em>. (No other pots currently contain plants.) For example, an initial state of <code>#..##....</code> indicates that pots <code>0</code>, <code>3</code>, and <code>4</code> currently contain plants.</p>
<p>Your puzzle input also contains some notes you find on a nearby table: someone has been trying to figure out how these plants <em>spread</em> to nearby pots.  Based on the notes, for each generation of plants, a given pot has or does not have a plant based on whether that pot (and the two pots on either side of it) had a plant in the last generation. These are written as <code>LLCRR =&gt; N</code>, where <code>L</code> are pots to the left, <code>C</code> is the current pot being considered, <code>R</code> are the pots to the right, and <code>N</code> is whether the current pot will have a plant in the next generation. For example:</p>
<ul>
<li>A note like <code>..#.. =&gt; .</code> means that a pot that contains a plant but with no plants within two pots of it will not have a plant in it during the next generation.</li>
<li>A note like <code>##.## =&gt; .</code> means that an empty pot with two plants on each side of it will remain empty in the next generation.</li>
<li>A note like <code>.##.# =&gt; #</code> means that a pot has a plant in a given generation if, in the previous generation, there were plants in that pot, the one immediately to the left, and the one two pots to the right, but not in the ones immediately to the right and two to the left.</li>
</ul>
<p>It's not clear what these plants are for, but you're sure it's important, so you'd like to make sure the current configuration of plants is sustainable by determining what will happen after <em><code>20</code> generations</em>.</p>
<p>For example, given the following input:</p>
<pre><code>initial state: #..#.#..##......###...###

...## =&gt; #
..#.. =&gt; #
.#... =&gt; #
.#.#. =&gt; #
.#.## =&gt; #
.##.. =&gt; #
.#### =&gt; #
#.#.# =&gt; #
#.### =&gt; #
##.#. =&gt; #
##.## =&gt; #
###.. =&gt; #
###.# =&gt; #
####. =&gt; #
</code></pre>

<p>For brevity, in this example, only the combinations which do produce a plant are listed. (Your input includes all possible combinations.) Then, the next 20 generations will look like this:</p>
<pre><code>                 1         2         3     
       0         0         0         0     
 0: ...#..#.#..##......###...###...........
 1: ...#...#....#.....#..#..#..#...........
 2: ...##..##...##....#..#..#..##..........
 3: ..#.#...#..#.#....#..#..#...#..........
 4: ...#.#..#...#.#...#..#..##..##.........
 5: ....#...##...#.#..#..#...#...#.........
 6: ....##.#.#....#...#..##..##..##........
 7: ...#..###.#...##..#...#...#...#........
 8: ...#....##.#.#.#..##..##..##..##.......
 9: ...##..#..#####....#...#...#...#.......
10: ..#.#..#...#.##....##..##..##..##......
11: ...#...##...#.#...#.#...#...#...#......
12: ...##.#.#....#.#...#.#..##..##..##.....
13: ..#..###.#....#.#...#....#...#...#.....
14: ..#....##.#....#.#..##...##..##..##....
15: ..##..#..#.#....#....#..#.#...#...#....
16: .#.#..#...#.#...##...#...#.#..##..##...
17: ..#...##...#.#.#.#...##...#....#...#...
18: ..##.#.#....#####.#.#.#...##...##..##..
19: .#..###.#..#.#.#######.#.#.#..#.#...#..
20: .#....##....#####...#######....#.#..##.
</code></pre>
<p>The generation is shown along the left, where <code>0</code> is the initial state.  The pot numbers are shown along the top, where <code>0</code> labels the center pot, negative-numbered pots extend to the left, and positive pots extend toward the right. Remember, the initial state begins at pot <code>0</code>, which is not the leftmost pot used in this example.</p>
<p>After one generation, only seven plants remain.  The one in pot <code>0</code> matched the rule looking for <code>..#..</code>, the one in pot <code>4</code> matched the rule looking for <code>.#.#.</code>, pot <code>9</code> matched <code>.##..</code>, and so on.</p>
<p>In this example, after 20 generations, the pots shown as <code>#</code> contain plants, the furthest left of which is pot <code>-2</code>, and the furthest right of which is pot <code>34</code>. Adding up all the numbers of plant-containing pots after the 20th generation produces <code><em>325</em></code>.</p>
<p><em>After <code>20</code> generations, what is the sum of the numbers of all pots which contain a plant?</em></p>
</article>


In [67]:
from re import split

from more_itertools import last


example = """
initial state: #..#.#..##......###...###

...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #
"""


def parse(input: str) -> tuple[str, set[str], set[str]]:
    initial_state, rules = split(r"(?:\r?\n){2,}", input.strip())
    initial_state = last(initial_state.split(": "))

    plant, no_plant = set(), set()

    for rule in rules.splitlines():
        llcrr, n = rule.split(" => ")
        if n == "#":
            plant.add(llcrr)
        else:
            no_plant.add(llcrr)

    return initial_state, plant, no_plant


def grow(input: str, generations: int = 20, should_print: bool = False) -> int:
    initial_state, plant, no_plant = parse(input)

    start, end = -3 * generations, 3 * generations + len(initial_state) + 1
    state = "..." * generations + initial_state + "..." * generations
    ws = list(range(start, end))
    if should_print:
        print(
            "    ",
            "  ".join(
                str((abs(i) // 100)) if i % 10 == 0 else " " for i in range(start, end)
            ),
        )
        print(
            "    ",
            "  ".join(
                str((abs(i) // 10) % 10) if i % 10 == 0 else " " for i in range(start, end)
            ),
        )
        print(
            "   ",
            " ".join(f"{'-' if i < 0 else ' '}{abs(i) % 10}" for i in range(start, end)),
        )
        print(f" 0  ", "  ".join(state))

    state1 = ["."] * len(state)
    for g in range(1, generations + 1):
        for i in range(len(state) - 5):
            subs = state[i : i + 5]
            if subs in plant:
                state1[i + 2] = "#"
            else:
                state1[i + 2] = "."

        state = "".join(state1)
        if should_print:
            print(f"{g:>2}   {"  ".join(state1)}")

    return sum(ws[i] for i in range(len(state)) if state[i] == "#")


print(f"Example {grow(example)} should be 325")

Example 325 should be 325


In [68]:
with open("../input/day12.txt") as f:
    puzzle = f.read()


print(f"Part I: {grow(puzzle)}")

Part I: 1917


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1917</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You realize that 20 generations aren't enough.  After all, these plants will need to last another 1500 years to even reach your timeline, not to mention your future.</p>
<p><em>After fifty billion (<code>50000000000</code>) generations, what is the sum of the numbers of all pots which contain a plant?</em></p>
</article>

</main>


In [69]:
from re import split

from more_itertools import last


example = """
initial state: #..#.#..##......###...###

...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #
"""


def parse(input: str) -> tuple[str, set[str], set[str]]:
    initial_state, rules = split(r"(?:\r?\n){2,}", input.strip())
    initial_state = last(initial_state.split(": "))

    plant, no_plant = set(), set()

    for rule in rules.splitlines():
        llcrr, n = rule.split(" => ")
        if n == "#":
            plant.add(llcrr)
        else:
            no_plant.add(llcrr)

    return initial_state, plant, no_plant


def grow(input: str, generation_x: int = 20) -> int:
    initial_state, plant, no_plant = parse(input)

    generations = 1_000
    start, end = -3 * generations, 3 * generations + len(initial_state) + 1
    state = "..." * generations + initial_state + "..." * generations
    ws = list(range(start, end))
    som = sum(ws[i] for i in range(len(state)) if state[i] == "#")

    diff = defaultdict(lambda: defaultdict(int))
    state1 = ["."] * len(state)

    for g in range(1, generations + 1):
        for i in range(len(state) - 5):
            subs = state[i : i + 5]
            if subs in plant:
                state1[i + 2] = "#"
            else:
                state1[i + 2] = "."

        state = "".join(state1)
        som1 = sum(ws[i] for i in range(len(state)) if state[i] == "#")
        diff[som1 - som][g] = som1
        som = som1

    m = max(diff.keys(), key=lambda k: len(diff[k]))
    sorted_gens = sorted(diff[m].keys())

    for i in range(len(sorted_gens) - 1):
        g = sorted_gens[i]
        if sorted_gens[i + 1] - g:
            del diff[m][g]
    g = min(diff[m].keys())
    v = diff[m][g]

    return v + m * (generation_x - g)


grow(example, generation_x=50_000_000_000)

999999999374

In [70]:
print(f"PartII: {grow(puzzle, generation_x=50_000_000_000)}")

PartII: 1250000000991


<link href="style.css" rel="stylesheet"></link>
<main class="read-aloud">

<p>Your puzzle answer was <code>1250000000991</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
